In [3]:
import os
import numpy as np
import pandas as pd
import cv2
from skimage import feature, measure
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops, label
from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image
from scipy.stats import kurtosis, skew
from sklearn.cluster import KMeans
from skimage.measure import shannon_entropy

# Define the function to calculate overlapping ratio
def calculate_overlapping_ratio(bboxes):
    num_boxes = len(bboxes)
    overlap_area = 0
    total_area = sum([w*h for x, y, w, h in bboxes])
    
    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            overlap_area += overlap(bboxes[i], bboxes[j])
    
    return overlap_area / total_area if total_area > 0 else 0

def overlap(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)
    
    intersection = max(0, xb - xa) * max(0, yb - ya)
    
    return intersection

# Create a list to store all image paths
root_folder = '/home/michael/Documenti/Milinda_Githubproject/FIDAC_1.0/'
categories = ['Tampered_Images_resize', 'Authentic Image_Rescieved']
all_images = []
labels = []

for category in categories:
    path = os.path.join(root_folder, category)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        if category == 'Tampered_Images_resize':
            labels.append(0)  # Fake = 0
        else:
            labels.append(1)  # Real = 1
        all_images.append(img_path)

# Initialize arrays to store the computed texture features
num_images = len(all_images)
mean_values = np.zeros(num_images)
variance_values = np.zeros(num_images)
homogeneity_values = np.zeros(num_images)
contrast_values = np.zeros(num_images)
dissimilarity_values = np.zeros(num_images)
entropy_values = np.zeros(num_images)
second_moment_values = np.zeros(num_images)
correlation_values = np.zeros(num_images)
energy_values = np.zeros(num_images)
std_deviation = np.zeros(num_images)
skewness_value = np.zeros(num_images)
kurtosis_value = np.zeros(num_images)
histogram_values = np.zeros(num_images)
difhihistogram_values = np.zeros(num_images // 2)
bdifhihistogram_values = np.zeros(num_images // 2)
area_values = np.zeros(num_images)
perimeter_values = np.zeros(num_images)
roughness_values = np.zeros(num_images)
average_gray_levels = np.zeros(num_images)
overlapping_ratios = np.zeros(num_images)

# Loop through the images and compute texture features
for i in range(num_images):
    # Read an image from the list
    img = cv2.imread(all_images[i])
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute texture features for the current image
    mean_values[i] = np.mean(gray_img)
    variance_values[i] = np.var(gray_img)
    
    # GLCM (Gray Level Co-occurrence Matrix)
    glcm = graycomatrix(gray_img, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
    homogeneity_values[i] = np.mean(graycoprops(glcm, 'homogeneity'))
    contrast_values[i] = np.mean(graycoprops(glcm, 'contrast'))
    correlation_values[i] = np.mean(graycoprops(glcm, 'correlation'))
    energy_values[i] = np.mean(graycoprops(glcm, 'energy'))
    entropy_values[i] = shannon_entropy(gray_img)
    second_moment_values[i] = np.sum((gray_img - mean_values[i]) ** 2) / gray_img.size
    std_deviation[i] = np.std(gray_img)
    skewness_value[i] = skew(gray_img.flatten())
    kurtosis_value[i] = kurtosis(gray_img.flatten())
    
    # Compute histogram
    hist, _ = np.histogram(gray_img, bins=256, range=(0, 256))
    histogram_values[i] = np.max(hist)

    # Area and perimeter
    threshold_value = threshold_otsu(gray_img)
    binary_img = gray_img > threshold_value
    area_values[i] = np.sum(binary_img)
    perimeter_img = measure.perimeter(binary_img)
    perimeter_values[i] = perimeter_img

    # Roughness approximation
    if area_values[i] > 0:
        equivalent_diameter = np.sqrt(4 * area_values[i] / np.pi)
        circle_perimeter = np.pi * equivalent_diameter
        roughness_values[i] = perimeter_values[i] / circle_perimeter

    # Average gray level
    average_gray_levels[i] = np.mean(gray_img)

    # MSER (Maximally Stable Extremal Regions) - detecting regions
    mser = cv2.MSER_create()
    regions, _ = mser.detectRegions(gray_img)
    bboxes = [cv2.boundingRect(p.reshape(-1, 1, 2)) for p in regions]
    
    # Calculate overlapping ratio
    overlapping_ratios[i] = calculate_overlapping_ratio(bboxes)

# Histogram difference features
for j in range(num_images // 2):
    difhihistogram_values[j] = histogram_values[j] - histogram_values[(num_images // 2) - 1 + j]
    bdifhihistogram_values[j] = histogram_values[(num_images // 2) - 1 + j] - histogram_values[j]

histogram_values = np.concatenate((difhihistogram_values, bdifhihistogram_values))

# Compile all features into a DataFrame
texture_table = pd.DataFrame({
    'ImageNum': np.arange(1, num_images + 1),
    'Label': labels,
    'Mean': mean_values,
    'Variance': variance_values,
    'Homogeneity': homogeneity_values,
    'Contrast': contrast_values,
    'Dissimilarity': dissimilarity_values,
    'Entropy': entropy_values,
    'SecondMoment': second_moment_values,
    'Correlation': correlation_values,
    'Energy': energy_values,
    'StdDeviation': std_deviation,
    'Skewness': skewness_value,
    'Kurtosis': kurtosis_value,
    'Histogram': histogram_values,
    'Area': area_values,
    'Perimeter': perimeter_values,
    'Roughness': roughness_values,
    'AverageGrayLevel': average_gray_levels,
    'OverlappingRatio': overlapping_ratios
})

# Save the table to a CSV file
texture_table.to_csv('/home/michael/Documenti/Milinda_Githubproject/Feartures_FIDAC_1.0.csv', index=False)

# Display the table
print(texture_table)



ValueError: All arrays must be of the same length

In [4]:
import os
import numpy as np
import pandas as pd
import cv2
from skimage import feature, measure
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops, label
from skimage.filters import threshold_otsu
from scipy.stats import kurtosis, skew
from skimage.measure import shannon_entropy

# Define the function to calculate overlapping ratio
def calculate_overlapping_ratio(bboxes):
    num_boxes = len(bboxes)
    overlap_area = 0
    total_area = sum([w*h for x, y, w, h in bboxes])
    
    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            overlap_area += overlap(bboxes[i], bboxes[j])
    
    return overlap_area / total_area if total_area > 0 else 0

def overlap(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)
    
    intersection = max(0, xb - xa) * max(0, yb - ya)
    
    return intersection

# Create a list to store all image paths
root_folder = '/home/michael/Documenti/Milinda_Githubproject/FIDAC_1.0/'
categories = ['Tampered_Images_resize', 'Authentic Image_Rescieved']
all_images = []
labels = []

for category in categories:
    path = os.path.join(root_folder, category)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        if category == 'Tampered_Images_resize':
            labels.append(0)  # Fake = 0
        else:
            labels.append(1)  # Real = 1
        all_images.append(img_path)

# Initialize arrays to store the computed texture features
num_images = len(all_images)
mean_values = np.zeros(num_images)
variance_values = np.zeros(num_images)
homogeneity_values = np.zeros(num_images)
contrast_values = np.zeros(num_images)
dissimilarity_values = np.zeros(num_images)
entropy_values = np.zeros(num_images)
second_moment_values = np.zeros(num_images)
correlation_values = np.zeros(num_images)
energy_values = np.zeros(num_images)
std_deviation = np.zeros(num_images)
skewness_value = np.zeros(num_images)
kurtosis_value = np.zeros(num_images)
histogram_values = np.zeros(num_images)
area_values = np.zeros(num_images)
perimeter_values = np.zeros(num_images)
roughness_values = np.zeros(num_images)
average_gray_levels = np.zeros(num_images)
overlapping_ratios = np.zeros(num_images)

# Loop through the images and compute texture features
for i in range(num_images):
    # Read an image from the list
    img = cv2.imread(all_images[i])
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute texture features for the current image
    mean_values[i] = np.mean(gray_img)
    variance_values[i] = np.var(gray_img)
    
    # GLCM (Gray Level Co-occurrence Matrix)
    glcm = graycomatrix(gray_img, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
    homogeneity_values[i] = np.mean(graycoprops(glcm, 'homogeneity'))
    contrast_values[i] = np.mean(graycoprops(glcm, 'contrast'))
    correlation_values[i] = np.mean(graycoprops(glcm, 'correlation'))
    energy_values[i] = np.mean(graycoprops(glcm, 'energy'))
    entropy_values[i] = shannon_entropy(gray_img)
    second_moment_values[i] = np.sum((gray_img - mean_values[i]) ** 2) / gray_img.size
    std_deviation[i] = np.std(gray_img)
    skewness_value[i] = skew(gray_img.flatten())
    kurtosis_value[i] = kurtosis(gray_img.flatten())
    
    # Compute histogram
    hist, _ = np.histogram(gray_img, bins=256, range=(0, 256))
    histogram_values[i] = np.max(hist)

    # Area and perimeter
    threshold_value = threshold_otsu(gray_img)
    binary_img = gray_img > threshold_value
    area_values[i] = np.sum(binary_img)
    perimeter_img = measure.perimeter(binary_img)
    perimeter_values[i] = perimeter_img

    # Roughness approximation
    if area_values[i] > 0:
        equivalent_diameter = np.sqrt(4 * area_values[i] / np.pi)
        circle_perimeter = np.pi * equivalent_diameter
        roughness_values[i] = perimeter_values[i] / circle_perimeter

    # Average gray level
    average_gray_levels[i] = np.mean(gray_img)

    # MSER (Maximally Stable Extremal Regions) - detecting regions
    mser = cv2.MSER_create()
    regions, _ = mser.detectRegions(gray_img)
    bboxes = [cv2.boundingRect(p.reshape(-1, 1, 2)) for p in regions]
    
    # Calculate overlapping ratio
    overlapping_ratios[i] = calculate_overlapping_ratio(bboxes)

# No need for histogram difference features; the issue was with those arrays being of different lengths

# Compile all features into a DataFrame
texture_table = pd.DataFrame({
    'ImageNum': np.arange(1, num_images + 1),
    'Label': labels,
    'Mean': mean_values,
    'Variance': variance_values,
    'Homogeneity': homogeneity_values,
    'Contrast': contrast_values,
    'Dissimilarity': dissimilarity_values,
    'Entropy': entropy_values,
    'SecondMoment': second_moment_values,
    'Correlation': correlation_values,
    'Energy': energy_values,
    'StdDeviation': std_deviation,
    'Skewness': skewness_value,
    'Kurtosis': kurtosis_value,
    'Histogram': histogram_values,
    'Area': area_values,
    'Perimeter': perimeter_values,
    'Roughness': roughness_values,
    'AverageGrayLevel': average_gray_levels,
    'OverlappingRatio': overlapping_ratios
})

# Save the table to a CSV file
texture_table.to_csv('/home/michael/Documenti/Milinda_Githubproject/Feartures_FIDAC_1.0.csv', index=False)

# Display the table
print(texture_table)


      ImageNum  Label        Mean     Variance  Homogeneity     Contrast  \
0            1      0  153.268101  2340.591961     0.266163   412.130517   
1            2      0  119.256477  2571.501298     0.187341   515.221850   
2            3      0   98.905102  1921.608724     0.173519   435.623517   
3            4      0   94.923325  4673.406471     0.223399   393.214325   
4            5      0  143.972443  1995.770282     0.400845   119.160158   
...        ...    ...         ...          ...          ...          ...   
1804      1805      1  116.562033  2402.600399     0.266661   839.974825   
1805      1806      1   46.557259  2114.668115     0.457573   278.888780   
1806      1807      1  122.619030  2422.130765     0.071638   566.795443   
1807      1808      1   98.765084  5268.339718     0.290885   371.432266   
1808      1809      1  111.709193  3156.348023     0.199176  1043.079525   

      Dissimilarity   Entropy  SecondMoment  Correlation    Energy  \
0               0